# Visualisierung der Coronazahlen in der Städteregion Aachen

In der folgenden Visualisierung erlaubt es, unterschiedliche Statistiken der Kommunen der Städteregion Aachen anzuzeigen, und zwischen Kommunen 

Auf dieser Seite werden die Daten von https://offenedaten.aachen.de/ verwendet.
Die Daten werden per Kommune angegeben, und auch für die gesamte Städteregion. Die Kommunen geben erst seit dem fünten August gesonderte Daten an, zuvor waren die Daten nur für die gesamte Städteregion verfügbar. Dementsprechend beginnen die Graphen über die einzelnen Kommunen zu diesem Zeitpunkt.

Bitte zum Laden der interaktiven Elemente alle Zellen neu ausführen.

In [1]:
import requests
import pandas as pd
import datetime
import seaborn as sns
sns.set_theme()
sns.set_palette("Paired")
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib notebook
from scipy.ndimage import gaussian_filter1d
from IPython.display import display,Image,HTML
from ipywidgets import interactive
import ipywidgets as widgets
import numpy as np
import json
figure_dims=(9, 6)

In [2]:
#load, process, smoothe, calculate

url = 'https://services-eu1.arcgis.com/2ypUQspLVcN0KOBE/arcgis/rest/services/CoronavirusFallzahlen_%C3%B6ffentlich/FeatureServer/1/query?where=1%3D1&outFields=Kommune,Meldedatum,Positiv,Genesen,Tote,Aktiv,Inzidenz,Creator&outSR=4326&f=json'
data = requests.get(url).json()
data=data["features"]
w_url = "https://services-eu1.arcgis.com/2ypUQspLVcN0KOBE/arcgis/rest/services/CoronavirusFallzahlen_%C3%B6ffentlich/FeatureServer/1/query?where=Kommune%20%3D%20%27W%C3%9CRSELEN%27&outFields=Kommune,Meldedatum,Positiv,Genesen,Tote,Aktiv,Inzidenz,Creator&outSR=4326&f=json"
w_data = requests.get(w_url).json()
w_data=w_data["features"]
combined = data+w_data
data_table = pd.DataFrame.from_dict({i:z for i,x in enumerate(combined) for y,z in x.items()},orient='index').drop(["Creator"],axis=1).drop_duplicates()
data_table["Meldedatum"] = data_table["Meldedatum"].apply(lambda d: datetime.datetime.fromtimestamp(int(d)/1000).strftime('%Y-%m-%d %H:%M:%S'))
data_table["Meldedatum"] = pd.to_datetime(data_table["Meldedatum"])
data_table["Tote_diff"] = data_table.groupby(['Kommune'])['Tote'].diff()
data_table["Tote_diff"].loc[data_table['Tote_diff'] < 1 ] = np.nan
data_table["Neuinfektionen"] = data_table.groupby(['Kommune'])["Positiv"].diff()
data_table["Neuinfektionen_7dm"] = data_table["Neuinfektionen"].rolling(7).mean()
data_table["Inzidenz_smoothed"] = gaussian_filter1d(data_table.Inzidenz,3)
data_table["last_7"] = data_table["Neuinfektionen"].rolling(7).sum()
data_table["last_4"] = data_table["Neuinfektionen"].rolling(4).sum()
data_table["r4"] = data_table["last_4"]/data_table["last_4"].shift(4)
data_table["r7"] = data_table["last_7"]/data_table["last_7"].shift(7)
data_table.set_index("Meldedatum",inplace=True)

/home/ulli/snap/jupyter/common/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
data_table[data_table.Kommune=="Würselen"].tail(10)

,Kommune,Positiv,Genesen,Tote,Aktiv,Inzidenz,Tote_diff,Neuinfektionen,Neuinfektionen_7dm,Inzidenz_smoothed,last_7,last_4,r4,r7
Meldedatum,,,,,,,,,,,,,,
2021-01-22 15:00:00,Würselen,1272,1132.0,54,86,67.086387,NaN,9.0,3.714286,118.257252,26.0,25.0,3.125000,0.317073
2021-01-23 15:00:00,Würselen,1285,1132.0,54,99,98.049334,NaN,13.0,5.428571,112.860641,38.0,30.0,3.333333,0.513514
2021-01-24 15:00:00,Würselen,1292,1144.0,54,94,116.111054,NaN,7.0,6.428571,110.332218,45.0,34.0,3.090909,0.652174
2021-01-25 15:00:00,Würselen,1293,1146.0,55,92,118.691299,1.0,1.0,6.571429,109.749558,46.0,30.0,1.875000,0.657143
2021-01-26 15:00:00,Würselen,1298,1147.0,56,95,110.950562,1.0,5.0,6.142857,109.967082,43.0,26.0,1.040000,0.614286
2021-01-27 15:00:00,Würselen,1303,1147.0,56,100,116.111054,NaN,5.0,6.428571,110.053001,45.0,18.0,0.600000,0.737705
2021-01-28 15:00:00,Würselen,1310,1149.0,56,105,121.271545,NaN,7.0,6.714286,109.555258,47.0,18.0,0.529412,1.958333
2021-01-29 15:00:00,Würselen,1317,1156.0,56,105,116.111054,NaN,7.0,6.428571,108.530584,45.0,24.0,0.800000,1.730769
2021-01-30 15:00:00,Würselen,1326,1167.0,56,103,105.790071,NaN,9.0,5.857143,107.392685,41.0,28.0,1.076923,1.078947


In [4]:
#load config
with open('config.json') as json_file: 
    config = json.load(json_file) 

In [5]:
def zeigekommune(kommune,statistik):
    """
    Plot one statistic per commune
    """
    if config[statistik]["plot_type"]=="lineplot": # config for line plot
        fig, g = plt.subplots(sharex=True,sharey=True,figsize=figure_dims)
        g = sns.lineplot(x="Meldedatum",y=statistik,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
    elif config[statistik]["plot_type"]=="scatterplot": # config for scatter plot
        g = sns.scatterplot(x="Meldedatum",y=statistik,color="r", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)

    g.yaxis.grid(True)
    g.set(ylabel=config[statistik]["desc"], xlabel='Meldedatum')
    plt.xticks(rotation=45)
    g.legend([kommune])

    plt.show()

In [6]:
u = interactive(zeigekommune, kommune=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Städteregion",
    rows=11,
    description='Kommune:',
),statistik=widgets.Dropdown(
    options=[(config[x]["desc"],x) for x in config],
    value="Inzidenz_smoothed",
    description='Statistik:',
    rows=len(config),
));
display(u)

interactive(children=(Dropdown(description='Kommune:', index=8, options=('Aachen', 'Alsdorf', 'Baesweiler', 'E…

In [7]:
def vergleichekommunen(kommune1,kommune2,statistik):
    """
Plot a given statical value for two communes on the same time scale
    """    
    fig, g = plt.subplots(sharex=True,sharey=True,figsize=figure_dims)
    if config[statistik]["plot_type"]=="lineplot": # config for line plot
        g = sns.lineplot(x="Meldedatum",y=statistik,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune1], ax=g)
        sns.lineplot(x="Meldedatum",y=statistik, color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune2], ax=g)
    elif config[statistik]["plot_type"]=="scatterplot": # config for scatter plot
        g = sns.scatterplot(x="Meldedatum",y=statistik,color="r", legend="full",
            data=data_table.loc[data_table.Kommune == kommune1], ax=g)
        sns.scatterplot(x="Meldedatum", y=statistik, color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune2], ax=g)

    g.yaxis.grid(True)
    g.set(ylabel=config[statistik]["desc"], xlabel='Meldedatum')
    g.legend([kommune1,kommune2])
    plt.xticks(rotation=45)
    plt.show()

In [8]:
w = interactive(vergleichekommunen, kommune1=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Aachen",
    rows=11,
    description='Kommune 1:',
),kommune2=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Alsdorf",
    rows=11,
    description='Kommune 2:',
),statistik=widgets.Dropdown(
    options=[(config[x]["desc"],x) for x in config],
    value="Inzidenz_smoothed",
    description='Statistik 2:',
    rows=len(config),
)
);
display(w)

interactive(children=(Dropdown(description='Kommune 1:', options=('Aachen', 'Alsdorf', 'Baesweiler', 'Eschweil…

In [9]:
def vergleichestatistik(kommune,statistik1,statistik2):
    """
    Plot two statistical values for a commune on the same time scale
    """
    if statistik2 in config[statistik1]["sharey"]:
        same_y=True
    else:
        same_y=False

    if same_y: # plot should have the same y axis labeling
        fig, g = plt.subplots(sharex=True,sharey=same_y, figsize=figure_dims)    
        if config[statistik1]["plot_type"]=="lineplot": # config for line plot
            g = sns.lineplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
            sns.lineplot(x="Meldedatum",y=statistik2,color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)
        elif config[statistik1]["plot_type"]=="scatterplot": # config for scatter plot
            g = sns.scatterplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
            sns.scatterplot(x="Meldedatum",y=statistik2,color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)
        g.legend([config[statistik1]["desc"], config[statistik2]["desc"]])



    else: # plot two different y axes
        fig, g = plt.subplots(sharex=True, figsize=figure_dims)    
        if config[statistik1]["plot_type"]=="lineplot": # config for line plot in statistic 1
            g = sns.lineplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
        elif config[statistik1]["plot_type"]=="scatterplot": #config for scatter plot in statistic 1
            g = sns.scatterplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
        h = g.twinx() # share x axis, two y axes
        if config[statistik2]["plot_type"]=="lineplot": # config for line plot in statistic 2
            h = sns.lineplot(x="Meldedatum",y=statistik2,color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=h)
        elif config[statistik2]["plot_type"]=="scatterplot": # config for scatter plot in statistic 2
            h = sns.scatterplot(x="Meldedatum",y=statistik2,color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=h)
        h.yaxis.grid(False)
        g.legend([config[statistik1]["desc"]])
        h.legend([config[statistik2]["desc"]])
        h.set(ylabel=config[statistik2]["unit"])


    g.set(ylabel=config[statistik1]["unit"], xlabel='Meldedatum')
    g.yaxis.grid(True)
    plt.xticks(rotation=45)

    plt.show()

In [10]:
v = interactive(vergleichestatistik, kommune=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Aachen",
    rows=11,
    description='Kommune:',
),statistik1=widgets.Dropdown(
    options=[(config[x]["desc"],x) for x in config],
    value="Inzidenz",
    rows=len(config),
    description='Statistik 1:',
),statistik2=widgets.Dropdown(
    options=[(config[x]["desc"],x) for x in config],
    value="Inzidenz_smoothed",
    description='Statistik 2:',
    rows=len(config),
)
);

In [11]:
display(v)

interactive(children=(Dropdown(description='Kommune:', options=('Aachen', 'Alsdorf', 'Baesweiler', 'Eschweiler…